In [1]:
import numpy as np
import h5py as h5
import torch
import cv2
from PIL import Image

## Testing some directory and OS stuff

In [9]:
# maybe very useful later for a script if we want to create a directory to save data 
import os
x = %pwd
path = 'testfolder'
test = os.path.join(x,path)
print(test)
os.mkdir(test)

C:\Users\richi\Documents\EPFL\Master\MA3_2\Machine Learning\practicals\projects\_project2\LOCAL_code\testfolder


In [11]:
test2 = os.path.join(os.getcwd(),'../testfolder')
print(test2)

C:\Users\richi\Documents\EPFL\Master\MA3_2\Machine Learning\practicals\projects\_project2\LOCAL_code\../testfolder


In [14]:
os.chdir('..')
print(os.getcwd())

C:\Users\richi\Documents\EPFL\Master\MA3_2\Machine Learning\practicals\projects\_project2


## Extracting all frames


In [ ]:
import cv2
# Opens the Video file
# Get your relative filepath for the video, should be something like ".../.../20200624_142043_crop_0_2_4.avi"
cap= cv2.VideoCapture(filepath)
outpath = '../misc/video/video_frames/frame_'
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite(outpath+str(i)+'.jpg',frame)
    i+=1
cap.release()
cv2.destroyAllWindows()

## Applying the Model 
(trained on Car images .... lol)


In [2]:
from __future__ import print_function, division
import os
import argparse
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from model.cnn_geometric_model import CNNGeometric
from data.pf_dataset import PFDataset
from data.download_datasets import download_PF_willow
from image.normalization import NormalizeImageDict, normalize_image
from util.torch_util import BatchTensorToVars, str_to_bool
from geotnf.transformation import GeometricTnf
from geotnf.point_tnf import *
import matplotlib.pyplot as plt
from skimage import io
import warnings
from torchvision.transforms import Normalize
from collections import OrderedDict

warnings.filterwarnings('ignore')

In [8]:
#Can be 'vgg' or 'resnet101'
model_aff_path = 'trained_models/base/aff-pascal-vgg/best_aff-pascal-vgg_affine_grid_lossvgg.pth.tar'
model_tps_path = 'trained_models/base/bestResnet101/best_checkpoint_adam_tps_grid_lossresnet101.pth.tar'
model_hom_path = ''

In [9]:
use_cuda = torch.cuda.is_available()
do_aff = not model_aff_path==''
do_tps = not model_tps_path==''
do_hom = not model_hom_path==''

# Create model
print('Creating CNN model...')

# CREATING MO
if do_aff:
    model_aff = CNNGeometric(output_dim=6,use_cuda=use_cuda,
                             feature_extraction_cnn='vgg')
if do_tps:
    model_tps = CNNGeometric(output_dim=18,use_cuda=use_cuda,
                             feature_extraction_cnn='resnet101')
#if do_hom:
#    if four_points:
#        model_hom = CNNGeometric(output_dim=8, use_cuda=use_cuda,
#                                feature_extraction_cnn=feature_extraction_cnn)
#    else : 
#        model_hom = CNNGeometric(output_dim=9, use_cuda=use_cuda,
#                        feature_extraction_cnn=feature_extraction_cnn)
# Load trained weights, not much to understand here, it's just that this is how it works.
print('Loading trained model weights...')
if do_aff:
    checkpoint = torch.load(model_aff_path, map_location=lambda storage, loc: storage)
    checkpoint['state_dict'] = OrderedDict([(k.replace('vgg', 'model'), v) for k, v in checkpoint['state_dict'].items()])
    model_aff.load_state_dict(checkpoint['state_dict'])
    print('aff done')
if do_tps:
    checkpoint = torch.load(model_tps_path, map_location=lambda storage, loc: storage)
    checkpoint['state_dict'] = OrderedDict([(k.replace('resnet', 'model'), v) for k, v in checkpoint['state_dict'].items()])
    model_tps.load_state_dict(checkpoint['state_dict'])
    print('tps done')

#Once again, while I wrote these lines, there is no model_homo_path trained so this is just for modularity
#if do_hom:
#    checkpoint = torch.load(model_hom_path, map_location=lambda storage, loc: storage)
#    checkpoint['state_dict'] = OrderedDict([(k.replace('vgg', 'model'), v) for k, v in checkpoint['state_dict'].items()])
#    model_hom.load_state_dict(checkpoint['state_dict'])

Creating CNN model...
Loading trained model weights...
aff done
tps done


In [10]:
tpsTnf = GeometricTnf(geometric_model='tps', use_cuda=use_cuda)
affTnf = GeometricTnf(geometric_model='affine', use_cuda=use_cuda)
homTnf = GeometricTnf(geometric_model='hom', use_cuda=use_cuda)

In [21]:
#Here resizes to 240x240 with bi-linear sampling
resizeCNN = GeometricTnf(out_h=240, out_w=240, use_cuda = False) 

def alt_preprocess_image(image,means,stds):
    """
    CALLED alt_preprocess_image because it differs a little bit from the one provided in the demo notebook
    """
    # convert to torch Variable
    image = np.expand_dims(image.transpose((2,0,1)),0)
    image = torch.Tensor(image.astype(np.float32)/255.0)
    image_var = Variable(image,requires_grad=False)
    # Resize image using bilinear sampling with identity affine tnf
    image_var = resizeCNN(image_var)
    # Normalize image
    image_var = normalize_image(image_var,mean=means,std=stds)
    return image_var

means=[0.485, 0.456, 0.406]
stds=[0.229, 0.224, 0.225]
normalizeTnf = Normalize(mean = means, std=stds)

In [22]:
def save_warped(source,target,name):
    #Redundant, reloads the image
    source_image = io.imread(source)
    target_image = io.imread(target)

    means=[0.485, 0.456, 0.406]
    stds=[0.229, 0.224, 0.225]
    #Use the preprocess method declared above to resize, normalize using the means and std computed above
    source_image_var = alt_preprocess_image(source_image,means=means, stds=stds)
    target_image_var = alt_preprocess_image(target_image,means=means, stds=stds)
    
    if use_cuda:
        source_image_var = source_image_var.cuda()
        target_image_var = target_image_var.cuda()
    #Create a "batch" (i.e. a pair) for the next cell below
    batch = {'source_image': source_image_var, 'target_image':target_image_var}
    #Resize target: create a function that will resize a given input into the target_image's dimension
    resizeTgt = GeometricTnf(out_h=target_image.shape[0], out_w=target_image.shape[1], use_cuda = use_cuda) 
    #Set the models to eval mode
    if do_aff:
        model_aff.eval()
    if do_tps:
        model_tps.eval()
    if do_hom:
        model_hom.eval()
        
    # Evaluate models and get the thetas
    if do_aff:
        theta_aff=model_aff(batch)
        warped_image_aff = affTnf(batch['source_image'],theta_aff.view(-1,2,3))
    
    if do_tps:
        theta_tps=model_tps(batch)
        warped_image_tps = tpsTnf(batch['source_image'],theta_tps)
    
    if do_aff and do_tps:
        theta_aff_tps=model_tps({'source_image': warped_image_aff, 'target_image': batch['target_image']})        
        warped_image_aff_tps = tpsTnf(warped_image_aff,theta_aff_tps)
    if do_aff:
        warped_image_aff_np = normalize_image(resizeTgt(warped_image_aff),forward=False).data.squeeze(0).transpose(0,1).transpose(1,2).cpu().numpy()
    
    if do_tps:
        warped_image_tps_np = normalize_image(resizeTgt(warped_image_tps),forward=False).data.squeeze(0).transpose(0,1).transpose(1,2).cpu().numpy()
    
    if do_aff and do_tps:
        warped_image_aff_tps_np = normalize_image(resizeTgt(warped_image_aff_tps),forward=False).data.squeeze(0).transpose(0,1).transpose(1,2).cpu().numpy()
    x = np.clip(warped_image_aff_tps_np,0,1)
    plt.imsave(name, x)
    



In [25]:
PATH = '../misc/video/'
framespath = PATH+'video_frames/frame_'
#outputpath = PATH+'iteration_0/'
target = framespath+'1117.jpg'
outpath = PATH+'aligned_to_1117/from_frame_'
for i in range(1715):
    source = framespath+str(num)+'.jpg'
    outname = outpath+str(num_1)+'to_frame_1117.jpg'
    save_warped(source,target,outname)

In [36]:
import glob
print(outpath)
img_array = []
for i in range(1715):
    filename = outpath+str(i)+'to_frame_1117.jpg'
    #print(filename)
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

img = cv2.imread(outpath+'0to_frame_1117.jpg')
h,w,l = img.shape
size = (w,h)
out = cv2.VideoWriter('test.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

../misc/video/aligned_to_1117/from_frame_
